## GPT2 Models for Json instructions
* Develop script for json instruction on raw GPT2 model
* Develop script for pre-training on json instructions
* Develop script for LORA fine-tuning on json instructions
* Learn how to download/load Llama, Store Llama in OneDrive
* Test Llama on json instruction
* Test fine-tuning Llama on json instruction
* Test code on GPU-enabled platforms

In [1]:
import os
import torch
import tiktoken

os.sys.path.append(r"C:\\Code\\SDev.Python") # Path to root above sdevpy
from sdevpy.llms import gpt
from sdevpy.llms import textgen as tg

# Global setup
model_source_folder = r"C:\\SDev.Finance\\OneDrive\\LLM\\models\\gpt2"
tokenizer = tiktoken.get_encoding("gpt2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load pre-trained model

In [2]:
model_size = "355M" # 124M, 355M, 774M, 1558M

# Retrieve parameters
model_folder = os.path.join(model_source_folder, model_size)
settings, params = gpt.load_gpt2(model_folder)
# print("Settings: ", settings)
# print("Param dict keys: ", params.keys())

# Create model
GPT_CONFIG = {"vocab_size": settings['n_vocab'], "context_length": settings['n_ctx'],
              "emb_dim": settings['n_embd'], "n_heads": settings['n_head'],
              "n_layers": settings['n_layer'], "drop_rate": 0.1, "qkv_bias": True}
context_length = GPT_CONFIG["context_length"]

model = gpt.GPTModel(GPT_CONFIG)
model.eval(); # Skip printing model details

# Load parameters into model
print("Loading weights for GPT-2 model size: " + model_size)
gpt.load_weights(model, params)
print("Done loading weights!")

# Send model to device
model.to(device); # Skip printing model details

Loading weights for GPT-2 model size: 355M
Done loading weights!


Check the model works by chatting with it. Choose token generator and initialize the ChatGenerator object, then use it to have a dialog.

In [3]:
torch.manual_seed(123)
token_gen = tg.NextTokenGenerator(top_k=15, temperature=1.5)
chat_gen = tg.ChatGenerator(device, model, tokenizer, context_length, token_gen,
                            max_new_tokens=50, max_sentences=2)

In [4]:
start_text = "Why is the President an idiot?"
print("Input text:\n", start_text)
print()
end_text = chat_gen.end_text(start_text)
print("Output text:\n", tg.format_answer(start_text, end_text))

Input text:
 Why is the President an idiot?

Output text:
 Why did he get elected to take us to War? Why are you fighting?


In [ ]:
new_text = "He didn't get elected to take us to War, did he?"
start_text = end_text + "\n" + new_text
print("Input text:\n", new_text)
print()
end_text = chat_gen.end_text(start_text)
print("Output text:\n", tg.format_answer(start_text, end_text))


Input text:
 He didn't get elected to take us to War, did he?

Output text:
 The reason they're not at war now is he didn't get in there with a military, that they didn't need anymore and there are some countries that we don't get into yet. He didn't get in there with a military.


Now using instructions

In [8]:
# Define json schema
schema = {"name": None, "age": None, "occupation": None, "email": None, "phone": None }
print("Schema: ", schema)

Schema:  {'name': None, 'age': None, 'occupation': None, 'email': None, 'phone': None}
